In [ ]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import TensorBoard

import sys
import io
from io import BytesIO
from google.cloud import storage
import urllib.request

import numpy as np
from numpy import random
from random import choices

from keras.models import load_model

def load_urls(list_urls):
  samples = []
  for url in list_urls:
    mp3file = urllib.request.urlopen(url)
    with open('./test.mp3','wb') as output:
      output.write(mp3file.read())
    song, sr = librosa.load(path = "./test.mp3", sr = 16000)
    samples.append(song)
  return samples

def mfcc(audios, label, INPUT_SHAPE):
  num_samples = len(audios)
  sr = 16000
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)

  count = 0
  for file in audios:
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
    mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
    mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
    data_X[count, :, :20] = mfcc.T[:126]
    data_X[count, :, 20:30] = mfcc_delta.T[:126]
    data_X[count, :, 30:] = mfcc_double_delta.T[:126]
    data_Y[count] = int(label)
    count += 1
  return data_X, data_Y

def fine_tune(model_path, x, y, ns, lr, gcp_key):
  randomize = np.arange(len(y))
  np.random.shuffle(randomize)
  X = x[randomize]
  Y = y[randomize]

  # X = np.expand_dims(X, axis=3)
  
  model = load_model(model_path)
  model.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(learning_rate = lr), metrics = ['acc', Precision()])
  model.fit(x = X, y = Y, epochs = ns)

  model.save("model.h5")
  storage_client = storage.Client.from_service_account_json(gcp_key)
  bucket = storage_client.get_bucket("user_model")
  filename = "new_model.h5"
  blob = bucket.blob(filename)
  with open('model.h5', 'rb') as f:
    blob.upload_from_file(f)
  print('Model fine-tuned and saved!')

if __name__ == "__main__":
  INPUT_SHAPE = (126,40)
  model_path = sys.argv[0] # Base model
  positives_urls = sys.argv[1] # List of positive urls
  negatives_urls = sys.argv[2] # List of negative urls
  n_steps = sys.argv[3] 
  learning_rate = sys.argv[4]
  gcp_key = sys.argv[5]

  positives = load_urls(positives_urls)
  positives_x, positives_y = mfcc(positives, True, INPUT_SHAPE)
  if len(negatives) > 0:
    negatives = load_urls(negatives_urls)
    negatives_x, negatives_y = mfcc(negatives, False, INPUT_SHAPE)
    final_x = np.concatenate(positives_x, negatives_x)
    final_y = np.concatenate(positives_y, negatives_y)
  else:
    final_x = positives_x
    final_y = positives_y

  fine_tune(model_path, final_x, final_y, n_steps, learning_rate, gcp_key)